# 作業2

In [5]:
import pandas as pd
import datetime as dt
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
import pickle

In [6]:
data = pd.read_csv('./LargeTrain.csv')
     
# get predictors and target titles
target ='Class'
predictors = [x for x in data.columns if x not in [target]]

# prepare training data and testing data
mask = np.random.rand(len(data)) < 0.8 
training = data[mask]
test = data[~mask]

# prepare training predictors and target
X = training[predictors]
y = training[target]

# initialize model
xgb_model = XGBClassifier(
    learning_rate = 0.1,
    n_estimators = 250,
    max_depth = 3,
    min_child_weight = 3,
    gamma = 0,
    subsample = 0.8,
    objective= 'binary:logistic',
    colsample_bytree = 0.8,
    nthread = 4,
    scale_pos_weight = 1,
    seed = 27)

## Step 1: 尋找 最佳的 max_depth 和 min_child_weight

In [7]:
param_test1 = {
    'max_depth':list(range(3,11,2)),
    'min_child_weight':list(range(3,11,2))  
}

gsearch1 = GridSearchCV(
    estimator = xgb_model,
    param_grid = param_test1,
    n_jobs = 4,
    iid = False,
    cv = 5)

gsearch1.fit(X, y)
gsearch1.grid_scores_

C:\Users\Eric\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.99687, std: 0.00054, params: {'min_child_weight': 3, 'max_depth': 3},
 mean: 0.99650, std: 0.00111, params: {'min_child_weight': 5, 'max_depth': 3},
 mean: 0.99650, std: 0.00161, params: {'min_child_weight': 7, 'max_depth': 3},
 mean: 0.99669, std: 0.00098, params: {'min_child_weight': 3, 'max_depth': 5},
 mean: 0.99660, std: 0.00138, params: {'min_child_weight': 5, 'max_depth': 5},
 mean: 0.99687, std: 0.00114, params: {'min_child_weight': 7, 'max_depth': 5},
 mean: 0.99696, std: 0.00095, params: {'min_child_weight': 3, 'max_depth': 7},
 mean: 0.99696, std: 0.00099, params: {'min_child_weight': 5, 'max_depth': 7},
 mean: 0.99678, std: 0.00142, params: {'min_child_weight': 7, 'max_depth': 7}]

In [13]:
print("best params:", gsearch1.best_params_, "best score: ", gsearch1.best_score_)

best params: {'min_child_weight': 5, 'max_depth': 7} best score:  0.996964285406


## Step 2: 尋找 最佳的 gamma

In [ ]:
# replace the best model to xgb_model
xgb_model = gsearch1.best_estimator_

param_test2 = {
 'gamma':[i/10.0 for i in range(0,5)]
}

gsearch2 = GridSearchCV(
    estimator = xgb_model,
    param_grid = param_test2,
    n_jobs = 4,
    iid = False,
    cv = 5)

gsearch2.fit(X, y)
gsearch2.grid_scores_